immoEliza project

libruries

In [4]:
# # Web Scraping

import pandas as pd
import requests
import re
import os
import time
from bs4 import BeautifulSoup


Web Scraping module

generate all urls

In [ ]:

class URLsScraperModule:
    # ------------------------------------------------------  generating 333 pages
    def __init__(self, first_page_url, total_group_pages, save_path_group_urls): 
        self.first_page_url = first_page_url
        self.total_group_pages = total_group_pages
        self.save_path = save_path_group_urls
        
        self.session = requests.Session()  # ایجاد یک Session برای مدیریت کوکی‌ها
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }  # هدر برای شبیه‌سازی درخواست از مرورگر واقعی

    def generate_page_links(self):
        """
        ایجاد لینک‌ها برای صفحات و ذخیره در فایل CSV.
        """
        links = []
        for page_num in range(1, self.total_group_pages + 1):
            page_group_utls = f"{self.first_page_url}&page={page_num}"
            links.append(page_group_utls)
        
        df = pd.DataFrame(links, columns=['page_group_utls'])
        df.to_csv(self.save_path, index=False)
              
        print(f"لینک‌های صفحات 1 تا {self.total_group_pages} در فایل {self.save_path} ذخیره شدند.")

    # ------------------------------------------------------ extract all links from a limited number of pages
    def extract_item_links(self, save_path_all_urls, limit=None):
        """
        استخراج لینک‌های آیتم‌های فروش از صفحات و ذخیره در فایل CSV.
        پارامتر `limit` برای تعیین تعداد صفحات استفاده می‌شود (مقدار None یعنی همه صفحات).
        """
        # خواندن لینک‌های صفحات از فایل CSV
        page_links_df = pd.read_csv(self.save_path)
        all_urls = []

        # اگر محدودیتی تعیین شده باشد، آن تعداد از صفحات انتخاب می‌شود
        if limit is not None:
            page_links_df = page_links_df.head(limit)

        # اسکرپ کردن از هر صفحه
        for page_url in page_links_df['page_group_utls']:
            try:
                response = self.session.get(page_url, headers=self.headers)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, 'html.parser')

                # استخراج لینک‌های آیتم‌ها
                for a_tag in soup.find_all('a', class_='card__title-link', href=True):
                    all_urls.append(a_tag['href'])
            except requests.RequestException as e:
                print(f"خطا در دسترسی به صفحه {page_url}: {str(e)}")
                continue

        # ذخیره لینک‌های آیتم‌ها در یک فایل CSV
        df = pd.DataFrame(all_urls, columns=['all_urls'])
        df.to_csv(save_path_all_urls, index=False)
        print(f"لینک‌های آیتم‌های فروش در فایل {save_path_all_urls} ذخیره شدند.")


# استفاده از کلاس و تولید لینک‌ها
first_page_url = "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&orderBy=relevance"
total_group_pages = 333
save_path_group_urls = "C:/Users/becod/AI/my-projects/my_immoweb/page_group_utls.csv"

scraper = URLsScraperModule(first_page_url, total_group_pages, save_path_group_urls)
scraper.generate_page_links()

# برای تست می‌توانید تعداد صفحات مورد نظر را تعیین کنید
save_path_all_urls = "C:/Users/becod/AI/my-projects/my_immoweb/all_urls.csv"
limit = 332                                                    # ///// to how many times run (limit is 1 - 333)
scraper.extract_item_links(save_path_all_urls, limit)  


Cleaning, step 1 (cleaning URLs, before scraping properties)

In [7]:
# URLs Cleaner Module

class URLsCleanerModule:
    
    # __________________________________________________________________________
    
    def __init__(self, input_file, output_file):
        """
        Initialize the URLsCleanerModule with input and output file paths.
        """
        self.input_file = input_file
        self.output_file = output_file
        self.df = None

    # __________________________________________________________________________
    
    # Function to read URLs from a CSV file
    def read_urls(self):
        """
        Reads URLs from a CSV file and loads them into a DataFrame.
        """
        try:
            self.df = pd.read_csv(self.input_file)
        except Exception as e:
            print(f"Error reading file {self.input_file}: {e}")
            self.df = None

    # __________________________________________________________________________
    
    # 1. Clean unnecessary whitespace from URLs
    def clean_whitespace(self):
        """
        Clean unnecessary whitespace from URLs in the DataFrame.
        """
        if self.df is not None:
            self.df['all_urls'] = self.df['all_urls'].str.strip()  # Strip leading/trailing whitespace
        return self

    # __________________________________________________________________________
    
    # 2. Check for broken URLs (validate format)
    def validate_urls(self):
        """
        Keep only valid URLs that start with 'http://' or 'https://'.
        """
        if self.df is not None:
            self.df = self.df[self.df['all_urls'].str.startswith(('http://', 'https://'))]
        return self

    # __________________________________________________________________________
    
    # 3. Remove URLs containing "new-real-estate-project-apartments" or "new-real-estate-project-houses"
    def remove_new_real_estate_projects(self):
        """
        Remove URLs containing 'new-real-estate-project-apartments' or 'new-real-estate-project-houses'.
        """
        if self.df is not None:
            self.df = self.df[~self.df['all_urls'].str.contains('new-real-estate-project-(?:apartments|houses)', case=False, regex=True)]
        return self

    # __________________________________________________________________________
    
    # 4. Remove duplicates from the URL dataset
    def remove_duplicates(self):
        """
        Remove duplicate URLs from the dataset.
        """
        if self.df is not None:
            self.df = self.df.drop_duplicates()
        return self

    # __________________________________________________________________________
    
    # Function to save cleaned URLs to a CSV file
    def save_cleaned_urls(self):
        """
        Save the cleaned URLs to a CSV file.
        """
        if self.df is not None:
            try:
                self.df.to_csv(self.output_file, index=False)
                print(f"Cleaned URLs saved to {self.output_file}")
            except Exception as e:
                print(f"Error saving file {self.output_file}: {e}")

    # __________________________________________________________________________
    
    # Main function for cleaning the URL dataset
    def clean_urls(self):
        """
        Orchestrates the entire URL cleaning process step by step.
        """
        self.read_urls()  # Read the URLs from the CSV file
        if self.df is None:
            return
        
        # Apply each cleaning function in sequence
        self.clean_whitespace()\
            .validate_urls()\
            .remove_new_real_estate_projects()\
            .remove_duplicates()\
            .save_cleaned_urls()

# __________________________________________________________________________

# Example usage: To run the cleaning process
if __name__ == "__main__":
    input_file = "C:/Users/becod/AI/my-projects/my_immoweb/all_urls.csv"
    output_file = "C:/Users/becod/AI/my-projects/my_immoweb/all_urls_cleaned.csv"
    
    # Create an instance of the URLsCleanerModule and run the cleaning pipeline
    cleaner = URLsCleanerModule(input_file, output_file)
    cleaner.clean_urls()


Cleaned URLs saved to C:/Users/becod/AI/my-projects/my_immoweb/all_urls_cleaned.csv


Scrape all properties from all cleaned URLs

In [ ]:

#_____________________________________________________________________

class ScraperModule:
    # Function to get the soup object from the webpage URLsCleanerModule
    def get_soup(self, url):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Check for HTTP errors
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
        except requests.exceptions.RequestException as e:
            print(f"Error fetching the URL {url}: {e}")
            return None
    #_____________________________________________________________________
    # extract Property ID from URL
    def get_property_id(self, url):
        try:
            return url.split('/')[-1]  # Last part of the URL
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract Postal Code from URL
    def get_postal_code(self, url):
        try:
            return url.split('/')[-2]  # Second-to-last part of the URL
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract Locality from URL
    def get_locality(self, url):
        try:
            return url.split('/')[-3]  # Third-to-last part of the URL
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract Sale Type from URL
    def get_sale_type(self, url):
        try:
            return url.split('/')[-4]  # Fourth-to-last part of the URL
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract Property Type from URL
    def get_property_type(self, url):
        try:
            return url.split('/')[-5]  # Fifth-to-last part of the URL
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    #  extract  address
    
    
    
    
    
    #_____________________________________________________________________
    # extract price 
    def get_price(self, soup):
        try:
            price_element = soup.find('p', class_='classified__price')
            if price_element:
                price_value = price_element.get_text(strip=True)

                # Check if there's a price range (min - max)
                if '-' in price_value:
                    min_price, max_price = price_value.split('-')
                    min_price = ''.join(filter(str.isdigit, min_price))  # Extract only digits for the min price
                    max_price = ''.join(filter(str.isdigit, max_price))  # Extract only digits for the max price
                    return f"{min_price} - {max_price}"  # Return the price range
                else:
                    # If there's only a single price
                    price = ''.join(filter(str.isdigit, price_value))  # Only keep digits
                    return price if price else "None"
            return "None"
        except Exception as e:
            return "None"

    #_____________________________________________________________________
    # extract living area 
    def get_living_area(self, soup):
        try:
            living_area_row = soup.find('th', string=lambda text: text and "Living area" in text)
            if living_area_row:
                living_area_value_tag = living_area_row.find_next('td', class_='classified-table__data')
                if living_area_value_tag:
                    for content in living_area_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the extracted number
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  number of bedrooms
    def get_bedrooms(self, soup):
        try:
            # Search for the section that contains "Bedrooms"
            bedrooms_row = soup.find('th', string=lambda text: text and "Bedrooms" in text)
            if bedrooms_row:
                bedrooms_value_tag = bedrooms_row.find_next('td', class_='classified-table__data')
                if bedrooms_value_tag:
                    # Loop through the contents of the tag to find the numeric value
                    for content in bedrooms_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the extracted number
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  number of bathrooms
    def get_bathrooms(self, soup):
        try:
            # Search for the section that contains "Bathrooms"
            bathrooms_row = soup.find('th', string=lambda text: text and "Bathrooms" in text)
            if bathrooms_row:
                bathrooms_value_tag = bathrooms_row.find_next('td', class_='classified-table__data')
                if bathrooms_value_tag:
                    # Loop through the contents of the tag to find the numeric value
                    for content in bathrooms_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the extracted number
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  kitchen type
    def get_kitchen_type(self, soup):
        try:
            # Search for the section that contains "Kitchen type"
            kitchen_row = soup.find('th', string=lambda text: text and "Kitchen type" in text)
            if kitchen_row:
                kitchen_value_tag = kitchen_row.find_next('td', class_='classified-table__data')
                if kitchen_value_tag:
                    # Loop through the contents to find the kitchen type (text string)
                    for content in kitchen_value_tag.contents:
                        if isinstance(content, str) and content.strip():
                            return content.strip()  # Return the kitchen type text
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  building condition 
    def get_building_condition(self, soup):
        try:
            # Search for the section that contains "Building condition"
            condition_row = soup.find('th', string=lambda text: text and "Building condition" in text)
            if condition_row:
                condition_value_tag = condition_row.find_next('td', class_='classified-table__data')
                if condition_value_tag:
                    # Loop through the contents to find the building condition (text string)
                    for content in condition_value_tag.contents:
                        if isinstance(content, str) and content.strip():
                            return content.strip()  # Return the building condition text
            return "None"
        except Exception as e:
            return "None"

    #_____________________________________________________________________
    # extract  number of frontages 
    def get_number_of_frontages(self, soup):
        try:
            # Search for the section that contains "Number of frontages"
            frontages_row = soup.find('th', string=lambda text: text and "Number of frontages" in text)
            if frontages_row:
                frontages_value_tag = frontages_row.find_next('td', class_='classified-table__data')
                if frontages_value_tag:
                    # Loop through the contents to find the frontages number (text string)
                    for content in frontages_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the number of frontages
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  number of floors 
    def get_number_of_floors(self, soup):
        try:
            # Search for the section that contains "Number of floors"
            floors_row = soup.find('th', string=lambda text: text and "Number of floors" in text)
            if floors_row:
                floors_value_tag = floors_row.find_next('td', class_='classified-table__data')
                if floors_value_tag:
                    # Loop through the contents to find the floors number (text string)
                    for content in floors_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the number of floors
            return "None"
        except Exception as e:
            return "None"

    #_____________________________________________________________________
    # extract floor number 
    def get_floor(self, soup):
        try:
            # Search for the section that contains "Floor"
            floor_row = soup.find('th', string=lambda text: text and "Floor" in text)
            if floor_row:
                floor_value_tag = floor_row.find_next('td', class_='classified-table__data')
                if floor_value_tag:
                    # Loop through the contents to find the floor number (text string)
                    for content in floor_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the floor number
            return "None"
        except Exception as e:
            return "None"

    #_____________________________________________________________________
    #  extract construction year
    def get_construction_year(self, soup):
        try:
            # Search for the section that contains "Construction year"
            year_row = soup.find('th', string=lambda text: text and "Construction year" in text)
            if year_row:
                year_value_tag = year_row.find_next('td', class_='classified-table__data')
                if year_value_tag:
                    # Loop through the contents to find the year (text string)
                    for content in year_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the construction year
            return "None"
        except Exception as e:
            return "None"   
    
    #_____________________________________________________________________
    # //////////  Interior
    # Fextract  Furnished status
    def get_furnished(self, soup):
        try:
            # Search for the section that contains "Furnished"
            furnished_row = soup.find('th', string=lambda text: text and "Furnished" in text)
            if furnished_row:
                furnished_value_tag = furnished_row.find_next('td', class_='classified-table__data')
                if furnished_value_tag:
                    # Loop through the contents of the tag to find the furnished status
                    for content in furnished_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned furnished status
            return "None"
        except Exception as e:
            return "None"

    #_____________________________________________________________________
    # Function to extract the Basement information from the soup object
    def get_basement(self, soup):
        try:
            # Search for the section that contains "Basement"
            basement_row = soup.find('th', string=lambda text: text and "Basement" in text)
            if basement_row:
                basement_value_tag = basement_row.find_next('td', class_='classified-table__data')
                if basement_value_tag:
                    # Loop through the contents of the tag to find the basement value
                    for content in basement_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned basement value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________    
    # //////////  Exterior
    # extract  Surface of the plot
    def get_surface_of_plot(self, soup):
        try:
            # Search for the section that contains "Surface of the plot"
            surface_row = soup.find('th', string=lambda text: text and "Surface of the plot" in text)
            if surface_row:
                surface_value_tag = surface_row.find_next('td', class_='classified-table__data')
                if surface_value_tag:
                    # Loop through the contents of the tag to find the surface value
                    for content in surface_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned surface value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________  
    # extract  Garden surface
    def get_garden_surface(self, soup):
        try:
            # Search for the section that contains "Garden surface"
            garden_surface_row = soup.find('th', string=lambda text: text and "Garden surface" in text)
            if garden_surface_row:
                garden_surface_value_tag = garden_surface_row.find_next('td', class_='classified-table__data')
                if garden_surface_value_tag:
                    # Loop through the contents of the tag to find the garden surface value
                    for content in garden_surface_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned garden surface value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  Terrace surface
    def get_terrace_surface(self, soup):
        try:
            # Search for the section that contains "Terrace surface"
            terrace_surface_row = soup.find('th', string=lambda text: text and "Terrace surface" in text)
            if terrace_surface_row:
                terrace_surface_value_tag = terrace_surface_row.find_next('td', class_='classified-table__data')
                if terrace_surface_value_tag:
                    # Loop through the contents of the tag to find the terrace surface value
                    for content in terrace_surface_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned terrace surface value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________ 
    # //////////  Facilities
    # extract  Elevator status
    def get_elevator(self, soup):
        try:
            # Search for the section that contains "Elevator"
            elevator_row = soup.find('th', string=lambda text: text and "Elevator" in text)
            if elevator_row:
                elevator_value_tag = elevator_row.find_next('td', class_='classified-table__data')
                if elevator_value_tag:
                    # Loop through the contents of the tag to find the elevator status
                    for content in elevator_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned elevator status
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract Swimming pool status
    def get_swimming_pool(self, soup):
        try:
            # Search for the section that contains "Swimming pool"
            swimming_pool_row = soup.find('th', string=lambda text: text and "Swimming pool" in text)
            if swimming_pool_row:
                swimming_pool_value_tag = swimming_pool_row.find_next('td', class_='classified-table__data')
                if swimming_pool_value_tag:
                    # Loop through the contents of the tag to find the swimming pool status
                    for content in swimming_pool_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned swimming pool status
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # //////////  Energy
    # extract  primary energy consumption 
    def get_primary_energy_consumption(self, soup):
        try:
            # Search for the section that contains "Primary energy consumption"
            energy_row = soup.find('th', string=lambda text: text and "Primary energy consumption" in text)
            if energy_row:
                energy_value_tag = energy_row.find_next('td', class_='classified-table__data')
                if energy_value_tag:
                    # Loop through the contents to find the value (text string)
                    for content in energy_value_tag.contents:
                        if isinstance(content, str) and content.strip().isdigit():
                            return content.strip()  # Return the energy consumption value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    # extract  Energy class
    def get_energy_class(self, soup):
        try:
            # Search for the section that contains "Energy class"
            energy_class_row = soup.find('th', string=lambda text: text and "Energy class" in text)
            if energy_class_row:
                energy_class_value_tag = energy_class_row.find_next('td', class_='classified-table__data')
                if energy_class_value_tag:
                    # Loop through the contents of the tag to find the energy class value
                    for content in energy_class_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned energy class value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    #/////////////////////////////////
    # extract Flood zone type 
    def get_flood_zone_type(self, soup):
        try:
            # Search for the section that contains "Flood zone type"
            flood_zone_row = soup.find('th', string=lambda text: text and "Flood zone type" in text)
            if flood_zone_row:
                flood_zone_value_tag = flood_zone_row.find_next('td', class_='classified-table__data')
                if flood_zone_value_tag:
                    # Loop through the contents of the tag to find the flood zone type value
                    for content in flood_zone_value_tag.contents:
                        if isinstance(content, str) and content.strip():  # Check if the content is a string and not empty
                            return content.strip()  # Return the cleaned flood zone type value
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    #/////////////////////////////////
    # extract Public sale 
    def get_public_sale(self, soup):
        try:
            # Search for the section that contains "Venue of the sale"
            venue_row = soup.find('th', string=lambda text: text and "Venue of the sale" in text)
            if venue_row:
                venue_value_tag = venue_row.find_next('td', class_='classified-table__data')
                if venue_value_tag:
                    # Check if there's a link indicating it's a public sale
                    link_tag = venue_value_tag.find(['a', 'iw-tracked-link'], href=True)
                    if link_tag:
                        return "Public sale"
            return "Not a public sale"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    #  extract price label 
    def get_price_label(self, soup):
        try:
            # Search for the section that contains the price label
            price_label_row = soup.find('p', class_='classified__price')
            if price_label_row:
                # Extract the text that could contain "Starting price" or "Make an offer"
                price_label = price_label_row.find('span', class_='classified__price-label')
                if price_label:
                    price_label_text = price_label.get_text(strip=True)

                    # Check if the label contains "Starting price"
                    if "Starting price" in price_label_text:
                        return "Starting price"

                    # Check if the label contains "Make an offer"
                    if "Make an offer" in price_label_text:
                        return "Make an offer"              
            return "None"
        except Exception as e:
            return "None"
    
    #_____________________________________________________________________
    #_____________________________________________________________________
    #_____________________________________________________________________
    # Function to process a single URL and extract all properties
    def scrape_property(self, url):
        soup = self.get_soup(url)
        if soup is None:  # If the page could not be fetched, return None
            return None

        property_details = {
            'Property ID': self.get_property_id(url),
            'Postal Code': self.get_postal_code(url),
            'Locality': self.get_locality(url),
            #'Address': self.get_address(url),
            'Property Type': self.get_property_type(url),
            'Sale Type': self.get_sale_type(url),
            'Price (€)': self.get_price(soup),
            'Surface of the plot (m²)': self.get_surface_of_plot(soup),
            'Living Area (m²)': self.get_living_area(soup),      
            'Bedrooms': self.get_bedrooms(soup),  
            'Bathrooms': self.get_bathrooms(soup),
            'Furnished': self.get_furnished(soup),
            'Kitchen Type': self.get_kitchen_type(soup),
            'Building condition': self.get_building_condition(soup),
            'number of Frontages': self.get_number_of_frontages(soup),
            'number of Floors': self.get_number_of_floors(soup),
            'Floor': self.get_floor(soup),
            'Construction Year': self.get_construction_year(soup),
            'Primary Energy Consumption (kWh/m²)': self.get_primary_energy_consumption(soup),
            'Energy class': self.get_energy_class(soup),
            'Flood Zone Type': self.get_flood_zone_type(soup),
            'Garden Surface': self.get_garden_surface(soup),
            'Terrace Surface': self.get_terrace_surface(soup),
            'Swimming Pool': self.get_swimming_pool(soup),
            'Elevator': self.get_elevator(soup), 
            'Basement': self.get_basement(soup),
            'Public Sale': self.get_public_sale(soup),
            'Price Label': self.get_price_label(soup),
            'URL': url  # Saving the URL with each property    
        } 
        return property_details
    
    #_____________________________________________________________________
    # Function to save property details to CSV
    def save_to_csv(self, property_list, filename):
        df = pd.DataFrame(property_list)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")
    
    #_____________________________________________________________________
    def read_urls_from_csv(self, file_path):
        try:
            df = pd.read_csv(file_path)
            return df[df.columns[0]].tolist()  # This assumes the first column contains URLs
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return []
    
    #_____________________________________________________________________
    # handle scraping for multiple URLs
    def scrape_multiple_properties(self, urls):
        all_properties = []

        for url in urls:
            property_details = self.scrape_property(url)

            if property_details:  # Only add the property details if they are valid
                all_properties.append(property_details)

            # Add a small delay to prevent rate-limiting
            time.sleep(1)  # 1-second delay between requests

        return all_properties
    
    #_____________________________________________________________________
    # Main execution
    def main(self):
        # File path for the CSV containing URLs
        url_file_path = "C:/Users/becod/AI/my-projects/my_immoweb/all_urls_cleaned.csv"

        # Reading the URLs from the CSV file
        urls = self.read_urls_from_csv(url_file_path)

        if not urls:
            print("No URLs found to process.")
            return

        # Scrape details for all URLs
        property_data = self.scrape_multiple_properties(urls)

        # Save the scraped data to a CSV file
        output_file_path = "C:/Users/becod/AI/my-projects/my_immoweb/property_details.csv"
        self.save_to_csv(property_data, output_file_path)

#_____________________________________________________________________
# Run the script
if __name__ == "__main__":
    scraper = ScraperModule()
    scraper.main()


Error reading file C:/Users/becod/AI/my-projects/my_immoweb/all_urls_cleaned.csv: name 'pd' is not defined
No URLs found to process.


Cleaning, step 2 (Cleaning Properties before Analyzing) 

In [11]:
# Cleaning Properties dataset

class PropertiesCleanerModule:

    # Reads a CSV file and returns a pandas DataFrame
    def read_csv(self, file_path):       
        if os.path.exists(file_path):
            return pd.read_csv(file_path)
        else:
            print(f"Error: File not found at {file_path}")
            return None

    # __________________________________________________________________________
    # 1. remove duplicate rows
    def remove_duplicates(self, data):
        if data.empty:
            print("Data is empty. Skipping duplicate removal.")
            return data
        return data.drop_duplicates()

    # __________________________________________________________________________
    # 2. remove rows with same properties and different 'Property ID' and 'URL' (remove 2 times registered items)
    def remove_duplicate_properties(self, data):
        # Check if 'Property ID' and 'URL' columns exist
        if 'Property ID' not in data.columns or 'URL' not in data.columns:
            print("'Property ID' or 'URL' column not found.")
            return data    
        # Define columns to compare, excluding 'Property ID' and 'URL'
        columns_to_compare = [col for col in data.columns if col not in ['Property ID', 'URL']]  
        # Find rows where all columns except 'Property ID' and 'URL' are duplicates
        duplicate_mask = data.duplicated(subset=columns_to_compare, keep=False)    
        # Keep only one row for each group of duplicates, regardless of 'Property ID' and 'URL'
        data_cleaned = data[~duplicate_mask | data.duplicated(subset=columns_to_compare, keep='first')]   
        return data_cleaned

    # __________________________________________________________________________
    # 3. remove rows if in column 'Public Sale' value is 'Public sale'    
    def remove_public_sale(self, data):
        if 'Public Sale' not in data.columns:
            print("'Public Sale' column not found.")
            return data  
        # Remove rows where 'Public Sale' column has the value 'Public sale'
        return data[~data['Public Sale'].isin(['Public sale'])]

    # __________________________________________________________________________

    # 4. remove rows if 'Price Label' value is "Starting price" or "Make an offer"
    def remove_price_label(self, data):
        # Check if 'Price Label' column exists
        if 'Price Label' not in data.columns:
            print("'Price Label' column not found.")
            return data 
        # Remove rows where 'Price Label' is "Starting price" or "Make an offer"
        return data[~data['Price Label'].isin(['Starting price', 'Make an offer'])]

    # __________________________________________________________________________
    # 3.1 Clean individual 'Locality name' by removing extra characters
    def clean_locality_name(self, name):
        name = name.replace('%C', 'e')               # replace  %C with  e (originally è)
        name = name.replace('%27', "'")              # replace  %27 with  '
        name = name.replace('%20', ' ')              # replace  %20 with  space
        name = re.sub(r'%20%28.*?%29', '', name)     # remove characters between %20%28 and %29
        name = re.sub(r'[0-9\(\)%]', '', name)       # remove any numbers and special characters
        name = re.sub(r'-\d+$', '', name)            # remove trailing numbers and hyphens
        name = name.lower().title()                  # lowercase, capitalize each word
        return name.strip()                          # strip extra spaces

    # 3.2 Apply the cleaning function to the 'Locality' column in the DataFrame
    def clean_locality_column(self, df):
        if 'Locality' in df.columns:
            df['Locality'] = df['Locality'].apply(self.clean_locality_name)  # Apply to each value in 'Locality'
        else:
            print("'Locality' column not found.")
        return df

    # __________________________________________________________________________

    # save the cleaned DataFrame to a CSV file
    def save_to_csv(self, data, output_path):
        if data is not None and not data.empty:
            data.to_csv(output_path, index=False)
            print(f"Data saved to {output_path}")
        else:
            print("No data to save.")

    # __________________________________________________________________________

    # main function to orchestrate the cleaning process
    def clean_data_pipeline(self, input_file, output_file):

        # Read the CSV file
        data = self.read_csv(input_file)

        # Check if data is available
        if data is None:
            print("No data found, exiting the pipeline.")
            return

        data_cleaned = self.remove_duplicates(data)
        data_cleaned = self.remove_duplicate_properties(data_cleaned)
        data_cleaned = self.remove_public_sale(data_cleaned) 
        data_cleaned = self.remove_price_label(data_cleaned) 
        data_cleaned = self.clean_locality_column(data_cleaned)

        # Save the cleaned data to a new CSV file
        self.save_to_csv(data_cleaned, output_file)

# __________________________________________________________________________

# Example usage: To run the cleaning process
if __name__ == "__main__":
    cleaner = PropertiesCleanerModule()
    input_file_path = "C:/Users/becod/AI/my-projects/my_immoweb/property_details.csv"
    output_file_path = "C:/Users/becod/AI/my-projects/my_immoweb/all_results_cleanedup.csv"
    
    # Run the cleaning pipeline
    cleaner.clean_data_pipeline(input_file_path, output_file_path)


Data saved to C:/Users/becod/AI/my-projects/my_immoweb/all_results_cleanedup.csv


In [ ]:
def print_page_html(item_url):
    """
    این متد برای چاپ HTML کامل یک صفحه استفاده می‌شود.
    """
    try:
        response = requests.get(item_url, headers={
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        })
        response.raise_for_status()
        
        # چاپ HTML کامل صفحه
        print(response.text)

    except requests.RequestException as e:
        print(f"خطا در دسترسی به آیتم {item_url}: {str(e)}")


# استفاده از تابع برای چاپ HTML یک صفحه
item_url = "https://www.immoweb.be/en/classified/house/for-sale/schaerbeek/1030/20260393"  # مثال از یک URL آیتم
print_page_html(item_url)


In [1]:
class WebScraperModule:
    def __init__(self, base_url):
        """
        این متد سازنده (constructor) است که URL وب‌سایت را ذخیره کرده و یک session برای مدیریت کوکی‌ها و درخواست‌ها ایجاد می‌کند.
        """
        self.base_url = base_url  # آدرس وب‌سایت
        self.session = None  # مدیریت کوکی‌ها و درخواست‌ها

    def access_site(self):
        """
        این متد مسئول دسترسی به سایت، مدیریت کوکی‌ها و خطاها است.
        """
        pass

    def extract_item_links(self):
        """
        این متد لینک‌های مربوط به آیتم‌های فروش را استخراج کرده و در یک فایل ذخیره می‌کند.
        """
        pass

    def extract_item_details(self):
        """
        این متد جزئیات آیتم‌ها را از لینک‌های ذخیره شده استخراج کرده و در یک فایل ذخیره می‌کند.
        """
        pass


Data Cleaning module


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# from scipy import stats

class DataCleaner:
    def __init__(self, data):
        self.data = data

    def remove_nulls(self):
        """Removes rows with null or invalid values."""
        # Code to clean data

    def normalize_data(self):
        """Normalizes and standardizes data (e.g., convert units)."""
        # Code to normalize data

Data Analysis & Visualization module


In [ ]:
class DataAnalyzer:
    def __init__(self, cleaned_data):
        self.cleaned_data = cleaned_data

    def analyze(self):
        """Performs data analysis and returns insights."""
        # Code to analyze the data

    def plot(self):
        """Generates charts and plots based on the analysis."""
        # Code to generate charts


Price Index module

Machine Learning module

Web Development module